# Commercial Impact Estimation Notebook - USI Social Impact
Last updated: 10 March, 2016 by [@DQOFFICIAL](https://github.com/DQOfficial)

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gp
from pandas import DataFrame
import math
import seaborn as sns
import sys
print sys.version
%pylab inline

2.7.11 (default, Dec  5 2015, 14:44:47) 
[GCC 4.2.1 Compatible Apple LLVM 7.0.0 (clang-700.1.76)]
Populating the interactive namespace from numpy and matplotlib


# PART 1 - Current Foot Traffic

## Ridership and foot traffic analysis:

In [4]:
# pull in files from turnstile notebook (usi_turnstile.ipynb)
metropolitan = pd.read_csv('station_metropolitan.csv')
lorimer = pd.read_csv('station_lorimer.csv')
nassau = pd.read_csv('station_nassau.csv')
bedford = pd.read_csv('station_bedford.csv')
graham = pd.read_csv('station_graham.csv')

In [5]:
Dictionary_stations={'metropolitan':metropolitan, 'lorimer':lorimer,'nassau':nassau,'bedford':bedford,'graham':graham}

In [6]:
av_daily_exits=pd.DataFrame(index=Dictionary_stations.keys(),columns=['count_daily_exits'])

# test for outliers in turnstile counts 
for st in Dictionary_stations:
    station=Dictionary_stations[st]
    station.rename(columns = {'Unnamed: 0':'turnstile'}, inplace = True) 
    station.set_index('turnstile',inplace = True)
    for date in station.columns:
        values=station[date]
        z_test=abs((values-mean(values)))<2.5*std(values) # use z-test
        outliers_indexes=z_test.index[z_test==False]
        if len(outliers_indexes!=0):
            print('OUTLIERS:',outliers_indexes,date,st)
            # replace the anomaly value with the median of that day
            station.replace(station.loc[outliers_indexes,date][0], np.median(station[date]), inplace = True)
    totals_list = [station[i].sum() for i in station.columns]
    av_daily_exits.count_daily_exits[st]=int(np.mean(totals_list))
    
    

('OUTLIERS:', Index([u'R35300-06-00'], dtype='object', name=u'turnstile'), '02/12/2016', 'lorimer')
('OUTLIERS:', Index([u'R35300-00-01'], dtype='object', name=u'turnstile'), '02/25/2016', 'lorimer')
('OUTLIERS:', Index([u'R23500-06-01'], dtype='object', name=u'turnstile'), '02/16/2016', 'bedford')


In [33]:
# calculate the precent of total exits for each station 
av_daily_exits['percent_daily_exits']=(av_daily_exits.count_daily_exits/np.sum(av_daily_exits.count_daily_exits))*100
av_daily_exits

,count_daily_exits,percent_daily_exits
nassau,3807,9.08527
lorimer,2553,6.09264
graham,5660,13.5074
metropolitan,3939,9.40028
bedford,25944,61.9144


Now, we are going to estimate the foot traffic in the area around Bedford Avenue. For that estimate, we consider a portion of the exits (expansion weight) on the other stations and the total of people comming out from Bedford.

We chose the negative exponencial function to model this expansion: this function has the value of 1 when the distance is 0 (that means, the weight is 1 in Bedford av) and the value vanishes quickly as the distance increases. 

In [8]:
### GRAPHIC OF THIS FUNCTION!!

In [9]:
import scipy.stats

In [10]:
# Euclidian distances from the Bedford Ave station calculated using Google Earth
distances_dic={'lorimer': 0.42,
'graham':0.7,
'metropolitan': 0.41 ,
'nassau': 0.58,
'bedford':0              }
distances=pd.DataFrame.from_dict(distances_dic,orient='index')
distances.rename(columns={0:'distance_to_bedford'},inplace= True)

In [34]:
# Create function to calcuate the expansion coefficient
# The expansion coefficient estimates the number of people traveling to our Bedford Ave radius based on the distance from Bedford Ave
def expansion(x,var):
    r=[]
    for i in x:
        r.append((scipy.stats.norm(0,var).pdf(i))/scipy.stats.norm(0,var).pdf(0))
    return r

In [101]:
# the expansion coefficient is the probability function of the normal distribution
# for the purposes of this study, we are using 0.3
expansion_coef=0.3

distances['expansion_proportion']=expansion(distances.distance_to_bedford,expansion_coef)
foot_traffic=av_daily_exits.join(distances)
foot_traffic

,count_daily_exits,percent_daily_exits,distance_to_bedford,expansion_proportion
nassau,3807,9.08527,0.58,0.154295
lorimer,2553,6.09264,0.42,0.375311
graham,5660,13.5074,0.70,0.065729
metropolitan,3939,9.40028,0.41,0.393022
bedford,25944,61.9144,0.00,1.000000


Then, the foot traffic is estimated as follows:

In [102]:
R_1=sum(foot_traffic.expansion_proportion*foot_traffic.count_daily_exits)

In [103]:
print('CONCLUSION: The total daily ridership today in the area around Bedford Av. station is equal to R=%d')%R_1

CONCLUSION: The total daily ridership today in the area around Bedford Av. station is equal to R=29409


# Part 2 - Expenditure in Williamsburg

- Total households in three Williamsburg zip codes (11206, 11211, and 11237)
- Average household expenture in Williamsburg
- Spend by category (listed below) that could contribute to GDP (categorical breakouts only available for all of NYC)

In [104]:
# households and average household anual expenditures for williamsburg zipcodes
# taken from http://www.point2homes.com/US/Neighborhood/NY/Brooklyn/Williamsburg-Demographics.html
households = 68787
total_expend = 40514

In [105]:
# estimate regional expenditure ammount 
Exp_wil = households * total_expend
print 'Estimated Total Household Expenditures in Williamsburg is: $',Exp_wil

Estimated Total Household Expenditures in Williamsburg is: $ 2786836518


However, we are interested in expenditures that might be associated with commercial activity in the area (for example: education or housing is not associated with commercial activity). 
We are considering expenditures in the following categories: 
 
 Healthcare = 5.6%
 
 Entertainment = 4.2% 
 
 Apparel = 3.9% 
 
 Dining = 13.2% 
 
 Other = 5.8%

In [106]:
# health = .056
# entertain = .042 
# apparel = .039 
# dining = .132 
# other = .058

category_names = ['healthcare', 'entertainment', 'apparel', 'dining','other']
categories = [.056, .042, .039, .132, .058]
my_list = [i*Exp_wil for i in categories] # multiply total expenditure by the percentage of relevant categories

#Commercial expenditures:
spend = pd.DataFrame(my_list, index= category_names)
spend.rename(columns = {0:'Spend $'}, inplace = True)
spend['Spend %'] = (spend['Spend $']/np.sum(spend['Spend $']))*100
spend

,Spend $,Spend %
healthcare,1.560628e+08,17.125382
entertainment,1.170471e+08,12.844037
apparel,1.086866e+08,11.926606
dining,3.678624e+08,40.366972
other,1.616365e+08,17.737003


In [107]:
# total relevant expenses amount is just the sum of all relevant categories
com_spend_W = sum(spend['Spend $'])
print 'Estimated Commercial spend in Williamsburg is: $',int(com_spend_W)
print 'Estimated Commercial spend is', (com_spend_W/Exp_wil)*100,'% of total estimated expenditures.'

Estimated Commercial spend in Williamsburg is: $ 911295541
Estimated Commercial spend is 32.7 % of total estimated expenditures.


# Part 3 - Calculate commercial space around the Bedford Ave. station
- Use PLUTO data
- Total commercial space for same three Williamsburg zip codes (11206, 11211, 11237)
- Commercial space within radius around the Bedford Ave. station, calculated using ArcGIS

In [108]:
#This file contains PLUTO data aggregated by census blocks around Bedford Av.
pluto_df=pd.read_csv('pluto_lehd_blocks.csv')
pluto_df.head()

,FID,bldgar,comare,resare,retail,office,zipco,geocode,C1,C2,C3,C4
0,0,456906,54500,402406,8000,28325,11211,360470517001001,40,6,0,0
1,1,180591,65390,115201,24730,20000,11211,360470517001002,82,133,1,0
2,2,227160,92068,135092,46914,32111,11211,360470517001003,227,77,0,5
3,3,100641,15010,85631,7440,6270,11211,360470517001004,20,159,0,0
4,4,156414,54939,101475,29310,0,11249,360470517002001,55,114,0,0


In [109]:
# Calculate commercial and retail density 
dC=(pluto_df['comare']+pluto_df['retail'])/pluto_df['bldgar']
pluto_df['commercial_density']=dC

In [110]:
ct_area=np.sum(pluto_df.comare)
# PLUTO: total Commerce area in Williamsburg:
total_comm = 39715979.0

# calculate the commercial area within our bedford ave radius
target_area = ct_area/total_comm

print 'Commercial square footage within radius is:', ct_area
print 'Total commercial square footage in Williamsburg is:', int(total_comm)
print 'The percentage of square footage within radius is:', target_area*100,'%'

Commercial square footage within radius is: 2595120
Total commercial square footage in Williamsburg is: 39715979
The percentage of square footage within radius is: 6.53419622364 %


In [111]:
# calculate commercial spend in our area, 
com_exp_bed = target_area * com_spend_W
com_exp_bed
print 'The estimated size of our economy around the Bedford Station is: $', int(com_exp_bed)

The estimated size of our economy around the Bedford Station is: $ 59545838


## Relationship Between Foot Traffic and Density of Comercial Areas

In [112]:
pluto_df.head()

,FID,bldgar,comare,resare,retail,office,zipco,geocode,C1,C2,C3,C4,commercial_density
0,0,456906,54500,402406,8000,28325,11211,360470517001001,40,6,0,0,0.136790
1,1,180591,65390,115201,24730,20000,11211,360470517001002,82,133,1,0,0.499028
2,2,227160,92068,135092,46914,32111,11211,360470517001003,227,77,0,5,0.611824
3,3,100641,15010,85631,7440,6270,11211,360470517001004,20,159,0,0,0.223070
4,4,156414,54939,101475,29310,0,11249,360470517002001,55,114,0,0,0.538628


In [113]:
pluto_df['alpha_coef']=pluto_df.commercial_density/(R_1/1000)
print "Relationship between comemrcial_density and foot traffic", mean(pluto_df.alpha_coef)

Relationship between comemrcial_density and foot traffic 0.0152138296899


# PART  4 - Estimating impact on foot traffic after tunnel closure

LEHD Data gave us the proportion of workers that might be affected for the Canarsie tunel shotdown (people traveling from Brooklyn to Manhattan every day) is 58%. 

So now, we are going to assume that our foot traffic is going to decrease for Bedford avenue to a 42% of the original value. The full explanation for this decision is explained in the paper.

In [114]:
#Change percent_daily_exits
impact_on_traf=0.58
#impact_on_traf=(3.0/2.0)*0.58

foot_traffic2=foot_traffic
foot_traffic2.head()
foot_traffic2.loc['bedford','count_daily_exits']=int(foot_traffic.loc['bedford','count_daily_exits']*impact_on_traf)
foot_traffic2

,count_daily_exits,percent_daily_exits,distance_to_bedford,expansion_proportion
nassau,3807,9.08527,0.58,0.154295
lorimer,2553,6.09264,0.42,0.375311
graham,5660,13.5074,0.70,0.065729
metropolitan,3939,9.40028,0.41,0.393022
bedford,15047,61.9144,0.00,1.000000


In [115]:
R_2=sum(foot_traffic.expansion_proportion*foot_traffic.count_daily_exits)
print('CONCLUSION: The forecasted daily foot traffic in the area around Bedford Av. station is equal to R = %d')%R_2

CONCLUSION: The forecasted daily foot traffic in the area around Bedford Av. station is equal to R = 18512


# Step 5 - Estimate Impact on Commercial Areas

Previously, the alpha coefficient (relationship between commercial density and foot traffic) was calculated for each census block.
Now, we are going to use the same alpha coefficient to calculate the possible reduction on commercial density

In [116]:
pluto_df2=pluto_df

In [121]:
# calculate the estimated impacts on commercial space after the shutdown
pluto_df2['projected_commercial_density']=pluto_df['commercial_density']*(R_2/1000)*pluto_df['alpha_coef']
pluto_df2['projected_commercial_area']=pluto_df2['projected_commercial_density']*pluto_df2['bldgar']
pluto_df2.head()

,FID,bldgar,comare,resare,retail,office,zipco,geocode,C1,C2,C3,C4,commercial_density,alpha_coef,projected_commercial_density,projected_commercial_area
0,0,456906,54500,402406,8000,28325,11211,360470517001001,40,6,0,0,0.136790,0.004651,0.011778,5381.612433
1,1,180591,65390,115201,24730,20000,11211,360470517001002,82,133,1,0,0.499028,0.016968,0.156758,28309.063838
2,2,227160,92068,135092,46914,32111,11211,360470517001003,227,77,0,5,0.611824,0.020803,0.235631,53525.964116
3,3,100641,15010,85631,7440,6270,11211,360470517001004,20,159,0,0,0.223070,0.007585,0.031323,3152.368592
4,4,156414,54939,101475,29310,0,11249,360470517002001,55,114,0,0,0.538628,0.018315,0.182624,28564.927850


In [122]:
pluto_df2.to_csv('pluto_coef.csv')

In [123]:
ct_area=np.sum(pluto_df.comare)
ct_area2=np.sum(pluto_df2.projected_commercial_area)
# PLUTO: total Commerce area in Williamsburg:

print 'Commercial square footage within radius today is:', ct_area
print'Commercial square footage within radius projected is:', ct_area2
print 'decrease of',(1-ct_area2/ct_area)

Commercial square footage within radius today is: 2595120
Commercial square footage within radius projected is: 1680688.53206
decrease of 0.3523657742


In [124]:
# calculate commercial spend in our area, 
total_comm = 39715979.0
target_area2 = ct_area2/total_comm
com_exp_bed_proj = target_area2 * com_spend_W
com_exp_bed=target_area * com_spend_W
print 'The estimated size of our economy around the Bedford Station is: $', int(com_exp_bed)
print 'The estimated size of our economy around the Bedford Station after closure of tunel: $', int(com_exp_bed_proj)
print 'decrease of',(1-com_exp_bed_proj/com_exp_bed)

The estimated size of our economy around the Bedford Station is: $ 59545838
The estimated size of our economy around the Bedford Station after closure of tunel: $ 38563923
decrease of 0.3523657742
